In [1]:
import numpy as np
from pathlib import Path
import xarray as xr

In [2]:
# path
path_data = Path("/home/yangliu/MLexpo/data")
erai_path = path_data / "ERA-Interim"
oras_path = path_data / "ORAS4"

In [3]:
start_year = 2000
end_year = 2017
# number of weeks per year
nweeks = 48
# load data
sic_xr = xr.load_dataset(erai_path / "sic_weekly_erai_1979_2017.nc")
sic_xr = sic_xr.sel(year=slice(start_year, end_year), latitude=slice(81, 63),
 longitude=slice(18, 59.5))
sic_xr

<xarray.Dataset>
Dimensions:    (year: 18, week: 48, latitude: 24, longitude: 56)
Coordinates:
  * year       (year) int32 2000 2001 2002 2003 2004 ... 2014 2015 2016 2017
  * week       (week) int32 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25
Data variables:
    sic        (year, week, latitude, longitude) float64 0.07345 ... -1.0
Attributes:
    description:  Weekly Sea Ice Concentration

In [4]:
# get mask from ORAS4 data
mask_ocean = xr.where(sic_xr['sic'][0,0,:,:] < 0, 1, 0)
mask_ocean.rename('mask')

<xarray.DataArray 'mask' (latitude: 24, longitude: 56)>
array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 1, 1]])
Coordinates:
    year       int32 2000
    week       int32 1
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25

In [5]:
sic_xr['mask'] = mask_ocean
sic_xr

<xarray.Dataset>
Dimensions:    (year: 18, week: 48, latitude: 24, longitude: 56)
Coordinates:
  * year       (year) int32 2000 2001 2002 2003 2004 ... 2014 2015 2016 2017
  * week       (week) int32 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25
Data variables:
    sic        (year, week, latitude, longitude) float64 0.07345 ... -1.0
    mask       (latitude, longitude) int64 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1
Attributes:
    description:  Weekly Sea Ice Concentration

In [6]:
sic_xr.to_netcdf('./data/sic_erai_2000_2017.nc')

In [7]:
ohc_xr = xr.load_dataset(oras_path / "ohc_monthly_oras2erai_1978_2017.nc")
# the coordinate name is wrong for week, it should be month
ohc_xr = ohc_xr.rename({'week': 'month'})
# 1 more year inlucded to calculate the relative difference
ohc_xr = ohc_xr.sel(year=slice(start_year - 1, end_year), latitude=slice(81, 63),
 longitude=slice(18, 59.5))
ohc_xr

<xarray.Dataset>
Dimensions:    (year: 19, month: 12, latitude: 24, longitude: 56)
Coordinates:
  * year       (year) int32 1999 2000 2001 2002 2003 ... 2014 2015 2016 2017
  * month      (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25
Data variables:
    OHC        (year, month, latitude, longitude) float64 -672.4 -672.4 ... 0.0
Attributes:
    description:  Monthly Ocean Heat Content regridded on rectilinear grid

In [8]:
sic_xr.longitude.data

array([18.  , 18.75, 19.5 , 20.25, 21.  , 21.75, 22.5 , 23.25, 24.  ,
       24.75, 25.5 , 26.25, 27.  , 27.75, 28.5 , 29.25, 30.  , 30.75,
       31.5 , 32.25, 33.  , 33.75, 34.5 , 35.25, 36.  , 36.75, 37.5 ,
       38.25, 39.  , 39.75, 40.5 , 41.25, 42.  , 42.75, 43.5 , 44.25,
       45.  , 45.75, 46.5 , 47.25, 48.  , 48.75, 49.5 , 50.25, 51.  ,
       51.75, 52.5 , 53.25, 54.  , 54.75, 55.5 , 56.25, 57.  , 57.75,
       58.5 , 59.25])

In [9]:
# turn dataset into sequences
ohc_seq_monthly = ohc_xr.OHC.values.reshape(-1, ohc_xr.latitude.size, ohc_xr.longitude.size)
# interpolation of ohc from monthly to weekly
ohc_diff_weekly = (ohc_seq_monthly[1:,:,:] - ohc_seq_monthly[:-1,:,:]) / 4
ohc_seq = np.zeros((sic_xr.year.size * nweeks, sic_xr.latitude.size, sic_xr.longitude.size),dtype=float)
# calculate relative difference, note that the first year data is edge case
for i in range(4):
    ohc_seq[3-i::4,:,:] = ohc_seq_monthly[12:,:,:] - i * ohc_diff_weekly[11:,:,:]

In [10]:
ohc_diff = xr.Dataset(
    data_vars=dict(
        ohc=(["year", "week", "latitude", "longitude"], ohc_seq.reshape(sic_xr.year.size,
         sic_xr.week.size, sic_xr.latitude.size, sic_xr.longitude.size)),
        mask=(["latitude", "longitude"], mask_ocean.data),
    ),
    coords=dict(
        year=(["year"], sic_xr.year.data),
        week=(["week"], sic_xr.week.data),
        latitude=(["latitude"], sic_xr.latitude.data),
        longitude=(["longitude"], sic_xr.longitude.data),
    ),
    attrs=dict(description="Weekly ocean heat content (300m) change (Tera Joule) regridded on rectilinear grid."),
)
ohc_diff

<xarray.Dataset>
Dimensions:    (year: 18, week: 48, latitude: 24, longitude: 56)
Coordinates:
  * year       (year) int32 2000 2001 2002 2003 2004 ... 2014 2015 2016 2017
  * week       (week) int32 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25
Data variables:
    ohc        (year, week, latitude, longitude) float64 108.1 108.1 ... 0.0 0.0
    mask       (latitude, longitude) int64 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1
Attributes:
    description:  Weekly ocean heat content (300m) change (Tera Joule) regrid...

In [11]:
ohc_diff.to_netcdf('./data/ohc_oras_2000_2017.nc')